# 데코레이터 사용하기

- 데코레이터는 프로그램의 버그를 찾는 디버깅, 함수의 성능 측정, 함수 실행 전에 데이터 확인 등에 활용합니다.

## 1. 데코레이터 만들기

- 데코레이터는 기존 함수를 수정하지 않으면서 추가 기능을 구현할 때 사용합니다.

In [20]:
def basic_latte(func):
    def wrapper():
        print('Milk')
        func()
        print('Espresso')
    return wrapper


def vanilla():
    print('Vanilla Syrup')
    
def caramel():
    print('Caramel Syrup')
    

vanilla_latte = basic_latte(vanilla)
vanilla_latte()
print()
caramel_latte = basic_latte(caramel)
caramel_latte()

Milk
Vanilla Syrup
Espresso

Milk
Caramel Syrup
Espresso


- @로 데코레이터 사용하기

```
@데코레이터
def 함수이름():
    코드
```

In [21]:
def basic_latte(func):
    def wrapper():
        print('Milk')
        func()
        print('Espresso')
    return wrapper


@basic_latte
def vanilla():
    print('Vanilla Syrup')

@basic_latte
def caramel():
    print('Caramel Syrup')
    

vanilla()
print()
caramel()

Milk
Vanilla Syrup
Espresso

Milk
Caramel Syrup
Espresso


### ※ 데코레이터를 여러 개 지정하기
- 함수 위에 데코레이터를 여러 줄로 지정해줍니다.
- 이때 데코레이터가 실행되는 순서는 위에서 아래 순입니다.

```
@데코레이터1
@데코레이터2
def 함수이름():
    코드
```

In [22]:
def decorator1(func):
    def wrapper():
        print('decorator1')
        func()
    return wrapper
 
def decorator2(func):
    def wrapper():
        print('decorator2')
        func()
    return wrapper
 
# 데코레이터를 여러 개 지정
@decorator1
@decorator2
def hello():
    print('hello')
 
hello()

decorator1
decorator2
hello


- @을 사용하지 않았을 때는 다음 코드와 동작이 같습니다.

In [23]:
def decorator1(func):
    def wrapper():
        print('decorator1')
        func()
    return wrapper
 
def decorator2(func):
    def wrapper():
        print('decorator2')
        func()
    return wrapper

def hello():
    print('hello')
 

decorated_hello = decorator1(decorator2(hello))
decorated_hello()

decorator1
decorator2
hello


## 2. 매개변수와 반환값을 처리하는 데코레이터 만들기

In [24]:
def trace(func):
    def wrapper(a, b):  # 호출할 함수 add(a, b)의 매개변수와 똑같이 지정
        r = func(a, b)  
        print('[{0}] a: {1}, b: {2} -> {3}'.format(func.__name__, a, b, r))
        return r
    return wrapper

@trace
def add(a, b):
    return a + b

print(add(1, 3))

[add] a: 1, b: 3 -> 4
4


- 가변 인수 함수 데코레이터

In [25]:
def trace(func):                     # 호출할 함수를 매개변수로 받음
    def wrapper(*args, **kwargs):    # 가변 인수 함수로 만듦
        r = func(*args, **kwargs)    # func에 args, kwargs를 언패킹하여 넣어줌
        print('{0}(args={1}, kwargs={2}) -> {3}'.format(func.__name__, args, kwargs, r))
                                     # 매개변수와 반환값 출력
        return r                     # func의 반환값을 반환
    return wrapper                   # wrapper 함수 반환
 
@trace                   # @데코레이터
def get_max(*args):      # 위치 인수를 사용하는 가변 인수 함수
    return max(args)
 
@trace                   # @데코레이터
def get_min(**kwargs):   # 키워드 인수를 사용하는 가변 인수 함수
    return min(kwargs.values())
 
print(get_max(10, 20))
print(get_min(x=10, y=20, z=30))

get_max(args=(10, 20), kwargs={}) -> 20
20
get_min(args=(), kwargs={'x': 10, 'y': 20, 'z': 30}) -> 10
10


### ※ 메서드에 데코레이터 사용하기

- 인스턴스 메서드는 항상 self를 받으므로 데코레이터를 만들 때도 wrapper 함수의 첫 번째 매개변수는 self로 지정해야 합니다(클래스 메서드는 cls)
- 마찬가지로 func를 호출할 때도 self와 매개변수를 그대로 넣어야 합니다.

In [26]:
def trace(func):
    def wrapper(self, a, b):   # 호출할 함수가 인스턴스 메서드이므로 첫 번째 매개변수는 self로 지정
        r = func(self, a, b)   # self와 매개변수를 그대로 넣어줌
        print('{0}(a={1}, b={2}) -> {3}'.format(func.__name__, a, b, r))   # 매개변수와 반환값 출력
        return r               # func의 반환값을 반환
    return wrapper
 
class Calc:
    @trace
    def add(self, a, b):    # add는 인스턴스 메서드
        return a + b
 
c = Calc()
print(c.add(10, 20))

add(a=10, b=20) -> 30
30


## 3. 매개변수가 있는 데코레이터 만들기

- 매개변수가 있는 데코레이터를 만들 때는 함수를 하나 더 만들어야 합니다.

In [30]:
def is_multiple(x):              # 데코레이터가 사용할 매개변수를 지정
    def real_decorator(func):    # 호출할 함수를 매개변수로 받음
        def wrapper(a, b):       # 호출할 함수의 매개변수와 똑같이 지정
            r = func(a, b)       # func를 호출하고 반환값을 변수에 저장
            if r % x == 0:       # func의 반환값이 x의 배수인지 확인
                print('{0}의 반환값은 {1}의 배수입니다.'.format(func.__name__, x))
            else:
                print('{0}의 반환값은 {1}의 배수가 아닙니다.'.format(func.__name__, x))
            return r             # func의 반환값을 반환
        return wrapper           # wrapper 함수 반환
    return real_decorator        # real_decorator 함수 반환
 
@is_multiple(3)     # @데코레이터(인수)
def add(a, b):
    return a + b
 
print(add(10, 20))
print(add(2, 5))

add의 반환값은 3의 배수입니다.
30
add의 반환값은 3의 배수가 아닙니다.
7


- 매개변수가 있는 데코레이터를 여러 개 지정하기

In [32]:
@is_multiple(3)
@is_multiple(7)
def add(a, b):
    return a + b
 
add(10, 20)

add의 반환값은 7의 배수가 아닙니다.
wrapper의 반환값은 3의 배수입니다.


30

### ※ 원래 함수 이름이 안나온다면?

- 함수의 원래 이름을 출력하고 싶다면 functools 모듈의 wraps 데코레이터를 사용해야 합니다. 다음과 같이 @functools.wraps에 func를 넣은 뒤 wrapper 함수 위에 지정해줍니다(from functools import wraps로 데코레이터를 가져왔다면 @wraps(func)를 지정).
- @functools.wraps는 원래 함수의 정보를 유지시켜줍니다. 따라서 디버깅을 할 때 유용하므로 데코레이터를 만들 때는 @functools.wraps를 사용하는 것이 좋습니다.

In [34]:
import functools


def is_multiple(x):
    def real_decorator(func):
        @functools.wraps(func)    # @functools.wraps에 func를 넣은 뒤 wrapper 함수 위에 지정
        def wrapper(a, b):
            r = func(a, b)
            if r % x == 0:
                print('{0}의 반환값은 {1}의 배수입니다.'.format(func.__name__, x))
            else:
                print('{0}의 반환값은 {1}의 배수가 아닙니다.'.format(func.__name__, x))
            return r
        return wrapper
    return real_decorator
 
@is_multiple(3)
@is_multiple(7)
def add(a, b):
    return a + b
 
add(10, 20)

add의 반환값은 7의 배수가 아닙니다.
add의 반환값은 3의 배수입니다.


30

## 4. 클래스로 데코레이터 만들기

- 클래스를 활용할 때는 인스턴스를 함수처럼 호출하게 해주는 \_\_call__ 메서드를 구현해야 합니다.

In [36]:
class Trace:
    def __init__(self, func):  # 호출할 함수를 인스턴스의 초깃값으로 받음
        self.func = func  # 호출할 함수를 속성 func에 저장
        
    def __call__(self):
        print(self.func.__name__, '함수 시작')
        self.func()
        print(self.func.__name__, '함수 끝')
        
@Trace  # 데코레이터
def hello():
    print('hello')
    
hello()

hello 함수 시작
hello
hello 함수 끝


- 클래스로 만든 데코레이터는 @을 지정하지 않고, 데코레이터의 반환값을 호출하는 방식으로도 사용할 수 있습니다.
- 클래스에 \_\_call__ 메서드를 정의했으므로 함수처럼 ( )(괄호)를 붙여서 호출할 수 있습니다.

In [38]:
def hello():    # @데코레이터를 지정하지 않음
    print('hello')
 
trace_hello = Trace(hello)    # 데코레이터에 호출할 함수를 넣어서 인스턴스 생성
trace_hello()                 # 인스턴스를 호출. __call__ 메서드가 호출됨

hello 함수 시작
hello
hello 함수 끝


## 5. 클래스로 매개변수와 반환값을 처리하는 데코레이터 만들기

In [39]:
class Trace:
    def __init__(self, func):    # 호출할 함수를 인스턴스의 초깃값으로 받음
        self.func = func         # 호출할 함수를 속성 func에 저장
 
    def __call__(self, *args, **kwargs):    # 호출할 함수의 매개변수를 처리
        r = self.func(*args, **kwargs) # self.func에 매개변수를 넣어서 호출하고 반환값을 변수에 저장
        print('{0}(args={1}, kwargs={2}) -> {3}'.format(self.func.__name__, args, kwargs, r))
                                            # 매개변수와 반환값 출력
        return r                            # self.func의 반환값을 반환
 
@Trace    # @데코레이터
def add(a, b):
    return a + b
 
print(add(10, 20))
print(add(a=10, b=20))

add(args=(10, 20), kwargs={}) -> 30
30
add(args=(), kwargs={'a': 10, 'b': 20}) -> 30
30


- 클래스로 매개변수가 있는 데코레이터 만들기

In [40]:
class IsMultiple:
    def __init__(self, x):         # 데코레이터가 사용할 매개변수를 초깃값으로 받음
        self.x = x                 # 매개변수를 속성 x에 저장
 
    def __call__(self, func):      # 호출할 함수를 매개변수로 받음
        def wrapper(a, b):         # 호출할 함수의 매개변수와 똑같이 지정(가변 인수로 작성해도 됨)
            r = func(a, b)         # func를 호출하고 반환값을 변수에 저장
            if r % self.x == 0:    # func의 반환값이 self.x의 배수인지 확인
                print('{0}의 반환값은 {1}의 배수입니다.'.format(func.__name__, self.x))
            else:
                print('{0}의 반환값은 {1}의 배수가 아닙니다.'.format(func.__name__, self.x))
            return r               # func의 반환값을 반환
        return wrapper             # wrapper 함수 반환
 
@IsMultiple(3)    # 데코레이터(인수)
def add(a, b):
    return a + b


print(add(10, 20))
print(add(2, 5))

add의 반환값은 3의 배수입니다.
30
add의 반환값은 3의 배수가 아닙니다.
7


## 42.8 심사문제 - HTML 태그 데코레이터 만들기

In [49]:
def html_tag(tag):
    def real_decorator(func):
        def wrapper():
            return '<' + tag + '>'+ func() + '</' + tag + '>'
        return wrapper
    return real_decorator


a, b = input().split()
 
@html_tag(a)
@html_tag(b)
def hello():
    return 'Hello, world!'
 
print(hello())

span p
<span><p>Hello, world!</p></span>
